In [1]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def prompt_to_chat(prompt, system = None):
    messages = [
        {"role": "user", "content": prompt}
    ]
    if system is not None:
        messages.insert(0, {"role": "system", "content": prompt})
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

def infer(model, prompt, l=100, use_chat = True, temperature=0.4, top_p = 0.8, system = None):
    if use_chat:
        prompt = prompt_to_chat(prompt, system)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=l,
        #temperature=temperature, 
        #top_p=top_p,
        do_sample=True ,  
        repetition_penalty = 1.1,
        temperature = 0.1,
        top_p = 0.3,
        top_k = 20,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
    return response

In [13]:

model_name = 'bb_ru_Cotype-Nano_20250123-100550'
model_path = f'/app/models/{model_name}'
base_model_name = 'MTSAIR/Cotype-Nano'
USE_QUANT = False

In [14]:
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_name)

In [5]:
if USE_QUANT:
    print("LOADING Q MODEL")
    bnb_config = transformers.BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    base_model = transformers.AutoModelForCausalLM.from_pretrained(base_model_name, device_map=device,quantization_config=bnb_config,)
else:
    base_model  = transformers.AutoModelForCausalLM.from_pretrained(
        base_model_name,
        low_cpu_mem_usage=True,
        return_dict=True,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )

In [6]:

model = peft.PeftModel.from_pretrained(base_model, model_path)

In [7]:
merged_model = model.merge_and_unload()
#merged_model = model

In [10]:
system = '''
Ты помощник для работы в системе BlackBox с использованием языка Component Pascal. Твоя задача информативно отвечать на вопросы. 
Ответ необходимо предоставить в формате markdown и выделять код символами ```. 

Пример: 

**Вопрос**: Как реализовать сортировку пузырьком?
**Ответ**: 

Cортировку пузырьком можно реализовать с помощью следующего кода:

```
	PROCEDURE BubbleSort*;  
		VAR i, j: INTEGER; x: Item;
	BEGIN
		FOR i := 1 TO n-1 DO
			FOR j := n-1 TO i BY -1 DO
				IF a[j-1] > a[j] THEN
					x := a[j-1];  a[j-1] := a[j];  a[j] := x
				END
			END
		END
	END BubbleSort;
```
'''

In [16]:
print(infer(merged_model, 'Как мне вывести массив в Log?', 512, use_chat = True, system = system))

Для начала, важно понять, что такое Log. Это текстовый журнал, который используется для отображения сообщений, которые могут быть важны для разработки или обслуживания программы. В частности, Log может использоваться для вывода информации о том, как программа выполняется.

Чтобы вывести массив в Log, вам нужно выполнить следующие шаги:

1. Открыть модуль, в котором находится массив.
2. Нажать Ctrl+H (или Shift+F9), чтобы перейти к его реализации.
3. Найти процедуру, которая использует этот массив.
4. Выбрать эту процедуру и нажать F5, чтобы запустить ее.
5. После завершения процедуры откройте Log, щелкнув по кнопке Log в меню Dev.
6. В Log будет показана информация о вызове процедуры.

Пример: предположим, что у нас есть следующий код на языке BlackBox:
	
	PROCEDURE Do*;
	BEGIN
		i := 0; j := 0;
		WHILE i < 10 DO
			j := 0;
			WHILE j < 10 DO
				Log.Int(i); Log.Int(j);
				Log.String(" "); INC(j)
			END
		END
	END Do;

Если выделите процедуру Do и нажмете F5, то получите следующее в L